### Exploring Customer Segmentation


<center>
    <img src = images/segments.jpeg>
</center>


In this activity, you are tasked with profiling customer groups for a large telecommunications company.  The data provided contains information on customers purchasing and useage behavior with the telecom products.  Your goal is to use PCA and clustering to segment these customers into meaningful groups, and report back your findings.  

Because these results need to be interpretable, it is important to keep the number of clusters reasonable.  Think about how you might represent some of the non-numeric features so that they can be included in your segmentation models.  You are to report back your approach and findings to the class.  Be specific about what features were used and how you interpret the resulting clusters.

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN

In [24]:
df = pd.read_csv('data/telco_churn_data.csv')

In [26]:
df.head()

,Customer ID,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,...,Latitude,Longitude,Population,Churn Value,CLTV,Churn Category,Churn Reason,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
0,8779-QRDMV,No,0,1,NaN,No,0.00,No,Yes,Fiber Optic,...,34.023810,-118.156582,68701,1,5433,Competitor,Competitor offered more data,5,0,NaN
1,7495-OOKFY,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Cable,...,34.044271,-118.185237,55668,1,5302,Competitor,Competitor made better offer,5,0,NaN
2,1658-BYGOY,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,...,34.108833,-118.229715,47534,1,3179,Competitor,Competitor made better offer,1,0,NaN
3,4598-XLKNJ,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,...,33.936291,-118.332639,27778,1,5337,Dissatisfaction,Limited range of services,1,1,2.0
4,4846-WHAFZ,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Cable,...,33.972119,-118.020188,26265,1,2793,Price,Extra data charges,1,0,2.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Referred a Friend                  7043 non-null   object 
 2   Number of Referrals                7043 non-null   int64  
 3   Tenure in Months                   7043 non-null   int64  
 4   Offer                              3166 non-null   object 
 5   Phone Service                      7043 non-null   object 
 6   Avg Monthly Long Distance Charges  7043 non-null   float64
 7   Multiple Lines                     7043 non-null   object 
 8   Internet Service                   7043 non-null   object 
 9   Internet Type                      5517 non-null   object 
 10  Avg Monthly GB Download            7043 non-null   int64  
 11  Online Security                    7043 non-null   objec

In [30]:
df.describe()

,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Regular Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Age,Number of Dependents,Zip Code,Latitude,Longitude,Population,Churn Value,CLTV,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,1834.000000
mean,1.951867,32.386767,22.958954,21.110890,65.538800,2280.381264,1.962182,278.499225,749.099262,46.509726,0.468692,93486.070567,36.197455,-119.756684,22139.603294,0.265370,4400.295755,1.338776,0.308107,3.005453
std,3.001199,24.542061,15.448113,20.948471,30.606805,2266.220462,7.902614,685.039625,846.660055,16.750352,0.962802,1856.767505,2.468929,2.154425,21152.392837,0.441561,1183.057152,1.430471,0.717514,1.256938
min,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,19.000000,0.000000,90001.000000,32.555828,-124.301372,11.000000,0.000000,2003.000000,0.000000,0.000000,1.000000
25%,0.000000,9.000000,9.210000,3.000000,35.890000,400.150000,0.000000,0.000000,70.545000,32.000000,0.000000,92101.000000,33.990646,-121.788090,2344.000000,0.000000,3469.000000,0.000000,0.000000,2.000000
50%,0.000000,29.000000,22.890000,17.000000,71.968000,1394.550000,0.000000,0.000000,401.440000,46.000000,0.000000,93518.000000,36.205465,-119.595293,17554.000000,0.000000,4527.000000,1.000000,0.000000,3.000000
75%,3.000000,55.000000,36.395000,28.000000,90.650000,3786.600000,0.000000,182.620000,1191.100000,60.000000,0.000000,95329.000000,38.161321,-117.969795,36125.000000,1.000000,5380.500000,2.000000,0.000000,4.000000
max,11.000000,72.000000,49.990000,94.000000,123.084000,8684.800000,49.790000,6477.000000,3564.720000,80.000000,9.000000,96150.000000,41.962127,-114.192901,105285.000000,1.000000,6500.000000,9.000000,6.000000,5.000000


In [48]:
#Drop unnecessary object columns
df_clean = df.drop(columns=['Customer ID', 'City', 'Payment Method', 'Churn Category', 'Churn Reason'])

# Replace empty strings or non-standard NaNs with actual NaNs for all columns
df_clean.replace('', np.nan, inplace=True)

# Drop rows where any column has NaN values
df_clean = df_clean.dropna()

#Code Yes/No columns as 1/0 to make them integer columns
yes_no_columns = [
    'Referred a Friend', 'Phone Service', 'Multiple Lines', 'Internet Service',
    'Online Security', 'Online Backup', 'Device Protection Plan',
    'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music',
    'Unlimited Data', 'Paperless Billing', 'Under 30', 'Senior Citizen',
    'Married', 'Dependents'
]

df_clean[yes_no_columns] = df_clean[yes_no_columns].replace({'Yes': 1, 'No': 0}).astype(int)

#Clean various columns by mapping the values to integers
offer_mapping = {'None': 0, 'Offer E': 5, 'Offer D': 4, 'Offer C': 3, 'Offer B': 2, 'Offer A': 1}
internet_type_mapping = {'Fiber Optic': 1, 'Cable': 2, 'DSL': 3, 'None': 0}
contract_mapping = {'Month-to-Month': 0, 'One Year': 1, 'Two Year': 2}
gender_mapping = {'Male': 1, 'Female': 2}

df_clean['Offer'] = df_clean['Offer'].map(offer_mapping)
df_clean['Internet Type'] = df_clean['Internet Type'].map(internet_type_mapping)
df_clean['Contract'] = df_clean['Contract'].map(contract_mapping)
df_clean['Gender'] = df_clean['Gender'].map(gender_mapping)

print(df_clean.isna().sum())
df_clean.info()
df_clean.shape

Referred a Friend                    0
Number of Referrals                  0
Tenure in Months                     0
Offer                                0
Phone Service                        0
Avg Monthly Long Distance Charges    0
Multiple Lines                       0
Internet Service                     0
Internet Type                        0
Avg Monthly GB Download              0
Online Security                      0
Online Backup                        0
Device Protection Plan               0
Premium Tech Support                 0
Streaming TV                         0
Streaming Movies                     0
Streaming Music                      0
Unlimited Data                       0
Contract                             0
Paperless Billing                    0
Monthly Charge                       0
Total Regular Charges                0
Total Refunds                        0
Total Extra Data Charges             0
Total Long Distance Charges          0
Gender                   

(685, 41)

In [52]:
df_scaled = (df_clean - df_clean.mean()) / df_clean.std()
df_scaled.head()

,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,...,Number of Dependents,Zip Code,Latitude,Longitude,Population,Churn Value,CLTV,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
3,1.102294,-0.300511,-0.258914,-0.156830,0.383652,-0.199339,-0.955026,0.0,-1.072643,-0.695723,...,0.579026,-1.672049,-0.884110,0.629169,0.303773,1.216438,0.774976,-0.803131,0.613195,-0.735669
4,1.102294,-0.300511,0.216997,-0.156830,0.383652,-1.037241,1.045563,0.0,0.124035,-0.599280,...,0.579026,-1.510215,-0.869465,0.776056,0.229016,1.216438,-1.438638,-0.803131,-0.576716,-0.735669
7,1.102294,1.397921,1.049842,-0.836757,-2.602727,-1.432171,-0.955026,0.0,0.124035,-0.068847,...,-0.430218,-1.281807,-0.789116,0.758513,-0.547062,1.216438,0.868080,-0.061696,0.613195,-1.538110
8,-0.905875,-0.640197,-0.655507,0.523097,0.383652,1.317369,1.045563,0.0,0.124035,-0.309953,...,1.588271,-1.237424,-0.801086,0.725063,0.104356,1.216438,1.105625,0.679739,-0.576716,-0.735669
10,-0.905875,-0.640197,-0.814145,0.523097,0.383652,-0.709068,-0.955026,0.0,1.320713,0.220480,...,0.579026,-0.967340,-0.773334,0.597491,1.754838,1.216438,1.334470,-0.061696,1.803106,-1.538110
